In [1]:
import os
import cv2
import pycolmap
from hloc.utils import viz_3d
import numpy as np
import pixtrack.utils.pose_utils as pose_utils
from pixloc.utils.colmap import Camera as ColCamera
from pixloc.pixlib.geometry import Camera as PixCamera, Pose
import matplotlib.pyplot as plt
from pixtrack.utils.ingp_utils import load_nerf2sfm, initialize_ingp, sfm_to_nerf_pose, nerf_to_sfm_pose
from pixtrack.visualization.run_vis_on_poses import get_nerf_image
from scipy.spatial.transform import Rotation as R

In [60]:
sfm_path = '/data/pixtrack/outputs/nerf_sfm/aug_gimble_04MAR2022/aug_sfm/'
model = pycolmap.Reconstruction(sfm_path)


nerf_path = '/data/pixtrack/instant-ngp/snapshots/gimble_04MAR2022/weights.msgpack'
nerf2sfm_path = '/data/pixtrack/pixel-perfect-sfm/datasets/gimble_04MAR2022/nerf2sfm.pkl'
nerf2sfm = load_nerf2sfm(nerf2sfm_path)
aabb_with_table = [[0.090, -0.493, -0.113], [0.895, 0.065, 0.714]] # this is for the gimbal
testbed = initialize_ingp(nerf_path)#, aabb=aabb_with_table)

05:21:25 INFO     Loading network config from: /data/pixtrack/instant-ngp/snapshots/gimble_04MAR2022/weights.msgpack
05:21:25 INFO     GridEncoding:  Nmin=16 b=1.44727 F=2 T=2^19 L=16
05:21:25 INFO     Density model: 3--[HashGrid]-->32--[FullyFusedMLP(neurons=64,layers=3)]-->1
05:21:25 INFO     Color model:   3--[Composite]-->16+16--[FullyFusedMLP(neurons=64,layers=4)]-->3
05:21:25 INFO       total_encoding_params=12599920 total_network_params=10240


In [61]:
camera = model.cameras[1]
camera = ColCamera(None, 
                camera.model_name,
                int(camera.width),
                int(camera.height),
                camera.params)
camera = PixCamera.from_colmap(camera)

In [62]:
all_imgs_in_altitude = []
for i in range(33, 73):
    all_imgs_in_altitude.append(f"mapping/IMG_95{i}.png")
render_poses = []
all_images = []
for image_name in all_imgs_in_altitude:
    image = model.find_image_with_name(image_name)
    cIw = pose_utils.get_camera_in_world_from_colmap_image(image)
    render_poses.append(sfm_to_nerf_pose(nerf2sfm, cIw))
    all_images_in_altitude.append(image)



In [63]:
!pip3 install mediapy

In [64]:
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp

In [65]:
import mediapy

In [90]:
testbed.nerf.sharpen = 1
testbed.background_color = [0.3, 0.3, 0.3, 0.3]
testbed.exposure = 0.2

In [91]:
image_renders = []
intermediate_frames = 3
#for pose_number in range(len(render_poses) - 1):
for pose_number in range(4):

    pose1 = render_poses[pose_number]
    pose2 = render_poses[pose_number+1]
    rotations = np.zeros((2, 3, 3))
    rotations[0, :, :] = pose1[:3, :3]
    rotations[1, :, :] = pose2[:3, :3]
    translations_x = np.linspace(pose1[0, -1], pose2[0, -1], intermediate_frames)
    translations_y = np.linspace(pose1[1, -1], pose2[1, -1], intermediate_frames)
    translations_z = np.linspace(pose1[2, -1], pose2[2, -1], intermediate_frames)

    interpolation_times = np.arange(0, intermediate_frames).tolist()
    print(interpolation_times)
    key_times = [0, intermediate_frames-1]    
    key_rotations = R.from_matrix(rotations)
    slerp = Slerp(key_times, key_rotations)
    intermediate_rotations = slerp(interpolation_times).as_matrix()
    #print(translations_x.shape[0])
    #import pdb; pdb.set_trace()

    for pose_num in range(translations_x.shape[0] - 1):
        camera_pose = np.eye(4)
        camera_pose[0, -1] = translations_x[pose_num]
        camera_pose[1, -1] = translations_y[pose_num]
        camera_pose[2, -1] = translations_z[pose_num]
        camera_pose[:3, :3] = intermediate_rotations[pose_num, :, :]
        nerf_img = get_nerf_image(testbed, camera_pose, camera)
        image_renders.append(nerf_img)


[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]


In [2]:
!pip3 install mediapy
import mediapy
mediapy.show_video(image_renders, width=320, height=240, fps=1)
#mediapy.write_video(images=image_renders, path="/data/pixtrack/outputs/gimbal_assets/without_chessboard.mp4")

NameError: name 'image_renders' is not defined

In [93]:
!mkdir /data/pixtrack/outputs/gimbal_assets

mkdir: cannot create directory ‘/data/pixtrack/outputs/gimbal_assets’: File exists
